# STEP 4 - Making DRL PySC2 Agent

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('..')
# pytorch network model을 정의한 디렉토리 (별도 작성)
# !unzip  skdrl.zip -d ~

## 0. Runnning 'Agent code' on jupyter notebook 

In [3]:
### unfortunately, PySC2 uses Abseil, which treats python code as if its run like an app
# This does not play well with jupyter notebook
# So we will need to monkeypatch sys.argv
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
#sys.argv = ["python", "--map", "AbyssalReef"]
sys.argv = ["python", "--map", "Simple64"]

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS-IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run an agent."""



import importlib
import threading

from absl import app
from absl import flags
from future.builtins import range  # pylint: disable=redefined-builtin

from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.lib import point_flag
from pysc2.lib import stopwatch
from pysc2.lib import actions

FLAGS = flags.FLAGS

# because of Abseil's horrible design for running code underneath Colabs
# We have to pull out this ugly hack from the hat
# print(globals())

if "flags_defined" not in globals():
    print("Flag set")
    flags.DEFINE_bool("render", False, "Whether to render with pygame.")
    point_flag.DEFINE_point("feature_screen_size", "84",
                            "Resolution for screen feature layers.")
    point_flag.DEFINE_point("feature_minimap_size", "64",
                            "Resolution for minimap feature layers.")
    point_flag.DEFINE_point("rgb_screen_size", None,
                            "Resolution for rendered screen.")
    point_flag.DEFINE_point("rgb_minimap_size", None,
                            "Resolution for rendered minimap.")
    flags.DEFINE_enum("action_space", "RAW", sc2_env.ActionSpace._member_names_,  # pylint: disable=protected-access
                      "Which action space to use. Needed if you take both feature "
                      "and rgb observations.")
    flags.DEFINE_bool("use_feature_units", False,
                      "Whether to include feature units.")
    flags.DEFINE_bool("use_raw_units", True,
                      "Whether to include raw units.")
    flags.DEFINE_integer("raw_resolution", 64, "Raw Resolution.")
    flags.DEFINE_bool("disable_fog", True, "Whether to disable Fog of War.")

    flags.DEFINE_integer("max_agent_steps", 0, "Total agent steps.")
    flags.DEFINE_integer("game_steps_per_episode", None, "Game steps per episode.")
    flags.DEFINE_integer("max_episodes", 0, "Total episodes.")
    flags.DEFINE_integer("step_mul", 8, "Game steps per agent step.")
    flags.DEFINE_float("fps", 22.4, "Frames per second to run the game.")

    #flags.DEFINE_string("agent", "sc2.agent.BasicAgent.ZergBasicAgent",
    #                    "Which agent to run, as a python path to an Agent class.")
    #flags.DEFINE_enum("agent_race", "zerg", sc2_env.Race._member_names_,  # pylint: disable=protected-access
    #                  "Agent 1's race.")
    flags.DEFINE_string("agent", "TerranRLAgentWithRawActsAndRawObs",
                        "Which agent to run, as a python path to an Agent class.")
    flags.DEFINE_enum("agent_race", "terran", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 1's race.")

    flags.DEFINE_string("agent2", "Bot", "Second agent, either Bot or agent class.")
    flags.DEFINE_enum("agent2_race", "zerg", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 2's race.")
#     flags.DEFINE_enum("difficulty", "very_easy", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
# The possible difficulty levels are very_easy, easy, medium, medium_hard, hard, harder, very_hard, cheat_vision, cheat_money and cheat_insane. 
# Please note that medium_hard is actually “hard” in the game,
    flags.DEFINE_enum("difficulty", "medium", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
                      "If agent2 is a built-in Bot, it's strength.")

    flags.DEFINE_bool("profile", False, "Whether to turn on code profiling.")
    flags.DEFINE_bool("trace", False, "Whether to trace the code execution.")
    flags.DEFINE_integer("parallel", 1, "How many instances to run in parallel.")

    flags.DEFINE_bool("save_replay", True, "Whether to save a replay at the end.")

    flags.DEFINE_string("map", None, "Name of a map to use.")
    flags.mark_flag_as_required("map")

flags_defined = True

# print(type(sc2_env.Race[FLAGS.agent2_race]))
# print(type(sc2_env.Difficulty[FLAGS.difficulty]))

def run_thread(agent_classes, players, map_name, visualize):
  """Run one thread worth of the environment with agents."""
  with sc2_env.SC2Env(
      map_name=map_name,
      players=players,
        agent_interface_format=features.AgentInterfaceFormat(
            action_space=actions.ActionSpace.RAW,
            use_raw_units=True,
            raw_resolution=64,
        ),
      step_mul=FLAGS.step_mul,
      game_steps_per_episode=FLAGS.game_steps_per_episode,
      disable_fog=FLAGS.disable_fog,
      visualize=visualize) as env:
    #env = available_actions_printer.AvailableActionsPrinter(env)
    agents = [agent_cls() for agent_cls in agent_classes]
    run_loop.run_loop(agents, env, FLAGS.max_agent_steps, FLAGS.max_episodes)
    if FLAGS.save_replay:
      env.save_replay(agent_classes[0].__name__)

def main(unused_argv):
  """Run an agent."""
  #stopwatch.sw.enabled = FLAGS.profile or FLAGS.trace
  #stopwatch.sw.trace = FLAGS.trace

  map_inst = maps.get(FLAGS.map)

  agent_classes = []
  players = []

  #agent_module, agent_name = FLAGS.agent.rsplit(".", 1)
  #agent_cls = getattr(importlib.import_module(agent_module), agent_name)
  #agent_classes.append(agent_cls)
  agent_classes.append(TerranRLAgentWithRawActsAndRawObs)
  players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent_race]))

  if map_inst.players >= 2:
    if FLAGS.agent2 == "Bot":
      print(FLAGS.agent2)   
      players.append(sc2_env.Bot(sc2_env.Race["terran"],
                                 sc2_env.Difficulty["medium"]))
    else:
      print("Not Bot")
      #agent_module, agent_name = FLAGS.agent2.rsplit(".", 1)
      #agent_cls = getattr(importlib.import_module(agent_module), agent_name)
      agent_classes.append(TerranRandomAgent)
      players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent2_race]))

  print(players)

  threads = []
  for _ in range(FLAGS.parallel - 1):
    t = threading.Thread(target=run_thread,
                         args=(agent_classes, players, FLAGS.map, False))
    threads.append(t)
    t.start()

  run_thread(agent_classes, players, FLAGS.map, FLAGS.render)

  for t in threads:
    t.join()

  if FLAGS.profile:
    pass
    #print(stopwatch.sw)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Flag set


## 3. Applying Vanilla DQN to a PySC2 Agent

구현된 기능

- Implementing 'Experience Replay' : 
    - 'Maximization Bias' 문제를 발생시키는 원인 중 하나인 'Sample간의 시간적 연관성'을 해결하기 위한 방법
    - Online Learning 에서 Batch Learning 으로 학습방법 바뀜 : Online update 는 Batch update 보다 일반적으로 Validation loss 가 더 높게 나타남.
    - Reinforcement Learning for Robots. Using Neural Networks. Long -Ji Lin. January 6, 1993. 논문에서 최초로 연구됨 http://isl.anthropomatik.kit.edu/pdf/Lin1993.pdf

- Implementing 'Fixed Q-Target' : 
    - 'Moving Q-Target' 문제 해결하기 위한 방법
    - 2015년 Nature 버전 DQN 논문에서 처음 제안됨. https://deepmind.com/research/publications/human-level-control-through-deep-reinforcement-learning 


구현되지 않은 기능

- Implementing 'Sensory Input Feature-Extraction' :
    - 게임의 Raw Image 를 Neural Net에 넣기 위한 Preprocessing(전처리) 과정
    - Raw Image 의 Sequence중 '최근 4개의 이미지'(과거 정보)를 하나의 새로운 State로 정의하여 non-MDP를 MDP 문제로 바꾸는 Preprocessing 과정 
    - CNN(합성곱 신경망)을 활용한 '차원의 저주' 극복

In [19]:
import random
import time
import math
import os.path

import numpy as np
import pandas as pd
from collections import deque
import pickle

from pysc2.agents import base_agent
from pysc2.env import sc2_env
from pysc2.lib import actions, features, units
from absl import app

import torch
from torch.utils.tensorboard import SummaryWriter

from skdrl.pytorch.model.mlp import NaiveMultiLayerPerceptron
from skdrl.common.memory.memory import ExperienceReplayMemory

# Network를 분리하였으므로, weight를 저정하는 공간도 2개로 분리

DATA_FILE_QNET = '30_rlagent_with_vanilla_dqn_qnet'
DATA_FILE_QNET_TARGET = '30_rlagent_with_vanilla_dqn_qnet_target'
SCORE_FILE = '30_rlagent_with_vanilla_dqn_score'

scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter('/home/jupyter/tensorboard_log')
print(device)



import torch
import torch.nn as nn
import numpy as np
import random

class DQN(nn.Module):

    def __init__(self,
                 state_dim: int,
                 action_dim: int,
                 qnet: nn.Module,
                 qnet_target: nn.Module,
                 lr: float,
                 gamma: float,
                 epsilon: float):
        """
        :param state_dim: input state dimension
        :param action_dim: action dimension
        :param qnet: main q network
        :param qnet_target: target q network
        :param lr: learning rate
        :param gamma: discount factor of MDP
        :param epsilon: E-greedy factor
        """

        super(DQN, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.qnet = qnet
        self.lr = lr
        self.gamma = gamma # discount future reward (미래의 보상을 조금만 반영)
        self.opt = torch.optim.Adam(params=self.qnet.parameters(), lr=lr)
        self.register_buffer('epsilon', torch.ones(1) * epsilon)

        # target network related
        qnet_target.load_state_dict(qnet.state_dict())
        self.qnet_target = qnet_target
        self.criteria = nn.SmoothL1Loss()

    def choose_action(self, state):
        qs = self.qnet(state)
        #prob = np.random.uniform(0.0, 1.0, 1)
        #if torch.from_numpy(prob).float() <= self.epsilon:  # random
        if random.random() <= self.epsilon: # random
            action = np.random.choice(range(self.action_dim))
        else:  # greedy
            action = qs.argmax(dim=-1)
        return int(action)

    def learn(self, state, action, reward, next_state, done):
        s, a, r, ns = state, action, reward, next_state

        # compute Q-Learning target with 'target network'
        with torch.no_grad():
            q_max, _ = self.qnet_target(ns).max(dim=-1, keepdims=True)
            q_target = r + self.gamma * q_max * (1 - done)

        q_val = self.qnet(s).gather(1, a)
        loss = self.criteria(q_val, q_target)

        self.opt.zero_grad()
        loss.backward()
        self.opt.step()


def prepare_training_inputs(sampled_exps, device='cpu'):
    states = []
    actions = []
    rewards = []
    next_states = []
    dones = []
    for sampled_exp in sampled_exps:
        states.append(sampled_exp[0])
        actions.append(sampled_exp[1])
        rewards.append(sampled_exp[2])
        next_states.append(sampled_exp[3])
        dones.append(sampled_exp[4])

    states = torch.cat(states, dim=0).float().to(device)
    actions = torch.cat(actions, dim=0).to(device)
    rewards = torch.cat(rewards, dim=0).float().to(device)
    next_states = torch.cat(next_states, dim=0).float().to(device)
    dones = torch.cat(dones, dim=0).float().to(device)
    return states, actions, rewards, next_states, dones


class TerranAgentWithRawActsAndRawObs(base_agent.BaseAgent):
    actions = ("do_nothing",
               "harvest_minerals",
               "build_supply_depot",
               "build_barracks",
               "train_marine",
               "attack")

    def get_my_units_by_type(self, obs, unit_type):
        return [unit for unit in obs.observation.raw_units
                if unit.unit_type == unit_type
                and unit.alliance == features.PlayerRelative.SELF]

    def get_enemy_units_by_type(self, obs, unit_type):
        return [unit for unit in obs.observation.raw_units
                if unit.unit_type == unit_type
                and unit.alliance == features.PlayerRelative.ENEMY]

    def get_my_completed_units_by_type(self, obs, unit_type):
        return [unit for unit in obs.observation.raw_units
                if unit.unit_type == unit_type
                and unit.build_progress == 100
                and unit.alliance == features.PlayerRelative.SELF]

    def get_enemy_completed_units_by_type(self, obs, unit_type):
        return [unit for unit in obs.observation.raw_units
                if unit.unit_type == unit_type
                and unit.build_progress == 100
                and unit.alliance == features.PlayerRelative.ENEMY]

    def get_distances(self, obs, units, xy):
        units_xy = [(unit.x, unit.y) for unit in units]
        return np.linalg.norm(np.array(units_xy) - np.array(xy), axis=1)

    def getMeanLocation(self, unitList):
        sum_x = 0
        sum_y = 0
        for unit in unitList:
            sum_x += unit.x
            sum_y += unit.y
        mean_x = sum_x / len(unitList)
        mean_y = sum_y / len(unitList)
        return [mean_x, mean_y]

    def transformDistance(self, x, x_distance, y, y_distance):
        if not self.base_top_left:
            return [x - x_distance, y - y_distance]
        return [x + x_distance, y + y_distance]

    def transformLocation(self, x, y):
        if not self.base_top_left:
            return [64 - x, 64 - y]
        return [x, y]

    def step(self, obs):
        super(TerranAgentWithRawActsAndRawObs, self).step(obs)
        if obs.first():
            command_center = self.get_my_units_by_type(
                obs, units.Terran.CommandCenter)[0]
            self.base_top_left = (command_center.x < 32)

    def do_nothing(self, obs):
        return actions.RAW_FUNCTIONS.no_op()

    def harvest_minerals(self, obs):
        scvs = self.get_my_units_by_type(obs, units.Terran.SCV)
        idle_scvs = [scv for scv in scvs if scv.order_length == 0]
        if len(idle_scvs) > 0:
            mineral_patches = [unit for unit in obs.observation.raw_units
                               if unit.unit_type in [
                                   units.Neutral.BattleStationMineralField,
                                   units.Neutral.BattleStationMineralField750,
                                   units.Neutral.LabMineralField,
                                   units.Neutral.LabMineralField750,
                                   units.Neutral.MineralField,
                                   units.Neutral.MineralField750,
                                   units.Neutral.PurifierMineralField,
                                   units.Neutral.PurifierMineralField750,
                                   units.Neutral.PurifierRichMineralField,
                                   units.Neutral.PurifierRichMineralField750,
                                   units.Neutral.RichMineralField,
                                   units.Neutral.RichMineralField750
                               ]]
            scv = random.choice(idle_scvs)
            distances = self.get_distances(obs, mineral_patches, (scv.x, scv.y))
            mineral_patch = mineral_patches[np.argmin(distances)]
            return actions.RAW_FUNCTIONS.Harvest_Gather_unit(
                "now", scv.tag, mineral_patch.tag)
        return actions.RAW_FUNCTIONS.no_op()

    def build_supply_depot(self, obs):
        supply_depots = self.get_my_units_by_type(obs, units.Terran.SupplyDepot)
        barrackses = self.get_my_units_by_type(obs, units.Terran.Barracks)
        barrack_ratio = (len(supply_depots) - len(barrackses) * 2)
        scvs = self.get_my_units_by_type(obs, units.Terran.SCV)
        if (obs.observation.player.minerals >= 100 and len(supply_depots) < 25):
            free_supply = (obs.observation.player.food_cap - obs.observation.player.food_used)
            exp_free_supply = (15 + len(supply_depots) * 8) - obs.observation.player.food_used
            # print("exp_cap : ",exp_free_supply, "cap : ", obs.observation.player.food_cap, " used: ", obs.observation.player.food_used)

            if (exp_free_supply <= 20):
#                 print("supply : ", len(supply_depots), "barrack : ", len(barrackses))
                supply_depot_xy = (22, 26) if self.base_top_left else (35, 42)
                ccs = self.get_my_units_by_type(obs, units.Terran.CommandCenter)
                if len(ccs) > 0:
                    mean_x, mean_y = self.getMeanLocation(ccs)
                    barracks_xy = (22, 21) if self.base_top_left else (35, 45)
                    if len(scvs) > 0:
                        scv = random.choice(scvs)
                        x = random.randint(mean_x - 15, mean_x + 15)
                        y = random.randint(mean_y - 10, mean_y + 10)
                        #                     print("build_supply", (x,y))
                        return actions.RAW_FUNCTIONS.Build_SupplyDepot_pt(
                            "now", scv.tag, (x, y))
        return actions.RAW_FUNCTIONS.no_op()

    def build_barracks(self, obs):
        supply_depots = self.get_my_units_by_type(obs, units.Terran.SupplyDepot)
        barrackses = self.get_my_units_by_type(obs, units.Terran.Barracks)
        scvs = self.get_my_units_by_type(obs, units.Terran.SCV)
        barrack_ratio = len(supply_depots) - len(barrackses)
        #         print("build barracks", obs.observation.player.minerals, barrack_ratio)
        if (obs.observation.player.minerals >= 150 and len(barrackses) < 15):
            if (len(supply_depots) > 0 and barrack_ratio >= 0):
                ccs = self.get_my_units_by_type(obs, units.Terran.CommandCenter)
                if len(ccs) > 0:
                    mean_x, mean_y = self.getMeanLocation(ccs)
                    if len(scvs) > 0:
                        scv = random.choice(scvs)
                        x = random.randint(mean_x - 15, mean_x + 15)
                        y = random.randint(mean_y - 12, mean_y + 12)
                        #                 print("build_barrack", )
                        return actions.RAW_FUNCTIONS.Build_Barracks_pt("now", scv.tag, (x, y))
        return actions.RAW_FUNCTIONS.no_op()

    def train_marine(self, obs):
        completed_barrackses = self.get_my_completed_units_by_type(obs, units.Terran.Barracks)
        free_supply = (obs.observation.player.food_cap - obs.observation.player.food_used)
        if (len(completed_barrackses) > 0 and obs.observation.player.minerals >= 100 and free_supply > 0):
            barracks = self.get_my_units_by_type(obs, units.Terran.Barracks)[0]
            if barracks.order_length < 5:
                return actions.RAW_FUNCTIONS.Train_Marine_quick("now", barracks.tag)
        return actions.RAW_FUNCTIONS.no_op()

    def attack(self, obs):
        marines = self.get_my_units_by_type(obs, units.Terran.Marine)
        if len(marines) > 3:

            attack_xy = (38, 44) if self.base_top_left else (19, 23)
            distances = self.get_distances(obs, marines, attack_xy)
            # print(distances)
            # print(np.argmax(distances))
            # print(marine)
            x_offset = random.randint(-12, 12)
            y_offset = random.randint(-7, 7)
            return actions.RAW_FUNCTIONS.Attack_pt(
                "now", [unit.tag for unit in marines], (attack_xy[0] + x_offset, attack_xy[1] + y_offset))
        return actions.RAW_FUNCTIONS.no_op()


class TerranRandomAgent(TerranAgentWithRawActsAndRawObs):
    def step(self, obs):
        super(TerranRandomAgent, self).step(obs)
        action = random.choice(self.actions)
        return getattr(self, action)(obs)

cuda:0


### Hyperparameter

하이퍼파라미터는 심층강화학습 알고리즘에서 성능에 매우 큰 영향을 미칩니다.
이 실험에 쓰인 하이퍼파라미터는 https://github.com/chucnorrisful/dqn 실험에서 제안된 값들을 참고하였습니다.
- self.epsilon = max(self.eps_min, self.eps_max - self.eps_min * (self.episode_count / 50))

In [ ]:
class TerranRLAgentWithRawActsAndRawObs(TerranAgentWithRawActsAndRawObs):
    def __init__(self):
        super(TerranRLAgentWithRawActsAndRawObs, self).__init__()

        self.s_dim = 12
        self.a_dim = 6

        self.lr = 1e-4 * 1
        self.batch_size = 32
        self.gamma = 0.99
        self.memory_size = 200000
        self.eps_max = 0.7
        self.eps_min = 0.02
        self.epsilon = 0.7
        self.init_sampling = 4000
        self.target_update_interval = 10

        self.data_file_qnet = DATA_FILE_QNET
        self.data_file_qnet_target = DATA_FILE_QNET_TARGET
        self.score_file = SCORE_FILE

        self.qnetwork = NaiveMultiLayerPerceptron(input_dim=self.s_dim,
                                                  output_dim=self.a_dim,
                                                  num_neurons=[256, 128, 64],
                                                  hidden_act_func='ReLU',
                                                  out_act_func='Identity').to(device)

        self.qnetwork_target = NaiveMultiLayerPerceptron(input_dim=self.s_dim,
                                                         output_dim=self.a_dim,
                                                         num_neurons=[256, 128, 64],
                                                         hidden_act_func='ReLU',
                                                         out_act_func='Identity').to(device)

        if os.path.isfile(self.data_file_qnet + '.pt'):
            self.qnetwork.load_state_dict(torch.load(self.data_file_qnet + '.pt', map_location=torch.device('cpu')))

        if os.path.isfile(self.data_file_qnet_target + '.pt'):
            self.qnetwork_target.load_state_dict(torch.load(self.data_file_qnet_target + '.pt', map_location=torch.device('cpu')))

        # initialize target network same as the main network.
        self.qnetwork_target.load_state_dict(self.qnetwork.state_dict())

        self.dqn = DQN(state_dim=self.s_dim,
                       action_dim=self.a_dim,
                       qnet=self.qnetwork,
                       qnet_target=self.qnetwork_target,
                       lr=self.lr,
                       gamma=self.gamma,
                       epsilon=self.epsilon).to(device)

        self.memory = ExperienceReplayMemory(self.memory_size)

        self.print_every = 1
        self.cum_reward = 0
        self.cum_loss = 0
        self.episode_count = 0

        self.new_game()

    def reset(self):
        super(TerranRLAgentWithRawActsAndRawObs, self).reset()
        self.new_game()

    def new_game(self):
        self.base_top_left = None
        self.previous_state = None
        self.previous_action = None
        self.cum_reward = 0
        self.cum_loss = 0

        # epsilon scheduling
        # slowly decaying_epsilon
        self.epsilon = max(self.eps_min, self.eps_max - self.eps_min * (self.episode_count / 50))
        self.dqn.epsilon = torch.tensor(self.epsilon).to(device)

    def get_state(self, obs):
        scvs = self.get_my_units_by_type(obs, units.Terran.SCV)
        idle_scvs = [scv for scv in scvs if scv.order_length == 0]
        command_centers = self.get_my_units_by_type(obs, units.Terran.CommandCenter)
        supply_depots = self.get_my_units_by_type(obs, units.Terran.SupplyDepot)
        completed_supply_depots = self.get_my_completed_units_by_type(
            obs, units.Terran.SupplyDepot)
        barrackses = self.get_my_units_by_type(obs, units.Terran.Barracks)
        completed_barrackses = self.get_my_completed_units_by_type(
            obs, units.Terran.Barracks)
        marines = self.get_my_units_by_type(obs, units.Terran.Marine)

        queued_marines = (completed_barrackses[0].order_length
                          if len(completed_barrackses) > 0 else 0)

        free_supply = (obs.observation.player.food_cap -
                       obs.observation.player.food_used)
        #         can_afford_supply_depot = obs.observation.player.minerals >= 100
        #         can_afford_barracks = obs.observation.player.minerals >= 150
        can_afford_marine = obs.observation.player.minerals >= 100
        too_much_minerals = obs.observation.player.minerals >= 2000
        minerals_size = round(obs.observation.player.minerals / 10, 1)

        enemy_scvs = self.get_enemy_units_by_type(obs, units.Terran.SCV)
        enemy_idle_scvs = [scv for scv in enemy_scvs if scv.order_length == 0]
        enemy_command_centers = self.get_enemy_units_by_type(
            obs, units.Terran.CommandCenter)
        enemy_supply_depots = self.get_enemy_units_by_type(
            obs, units.Terran.SupplyDepot)
        enemy_completed_supply_depots = self.get_enemy_completed_units_by_type(
            obs, units.Terran.SupplyDepot)
        enemy_barrackses = self.get_enemy_units_by_type(obs, units.Terran.Barracks)
        enemy_completed_barrackses = self.get_enemy_completed_units_by_type(
            obs, units.Terran.Barracks)
        enemy_Factory = self.get_enemy_units_by_type(obs, units.Terran.Factory)
        enemy_Starport = self.get_enemy_units_by_type(obs, units.Terran.Starport)
        enemy_Bunker = self.get_enemy_units_by_type(obs, units.Terran.Bunker)

        enemy_marines = self.get_enemy_units_by_type(obs, units.Terran.Marine)
        enemy_Marauder = self.get_enemy_units_by_type(obs, units.Terran.Marauder)
        enemy_Reaper = self.get_enemy_units_by_type(obs, units.Terran.Reaper)
        enemy_Hellion = self.get_enemy_units_by_type(obs, units.Terran.Hellion)
        enemy_Hellbat = self.get_enemy_units_by_type(obs, units.Terran.Hellbat)
        enemy_SiegeTank = self.get_enemy_units_by_type(obs, units.Terran.SiegeTank)
        enemy_Cyclone = self.get_enemy_units_by_type(obs, units.Terran.Cyclone)
        enemy_WidowMine = self.get_enemy_units_by_type(obs, units.Terran.WidowMine)
        enemy_Thor = self.get_enemy_units_by_type(obs, units.Terran.Thor)
        enemy_Viking = self.get_enemy_units_by_type(obs, units.Terran.VikingAssault)
        enemy_Medivac = self.get_enemy_units_by_type(obs, units.Terran.Medivac)
        enemy_Liberator = self.get_enemy_units_by_type(obs, units.Terran.Liberator)
        enemy_Raven = self.get_enemy_units_by_type(obs, units.Terran.Raven)
        enemy_Battlecruiser = self.get_enemy_units_by_type(obs, units.Terran.Battlecruiser)
        enemy_land_count = len(enemy_marines) + len(enemy_Marauder) + len(enemy_Reaper) + len(enemy_Hellion) + \
                           len(enemy_Hellbat) + len(enemy_SiegeTank) + len(enemy_Cyclone) + len(enemy_WidowMine) + len(enemy_Thor)
        enemy_air_count = len(enemy_Viking) + len(enemy_Medivac) + len(enemy_Medivac) + len(enemy_Liberator) + len(enemy_Raven) + len(enemy_Battlecruiser)
        enemy_total_count = enemy_land_count + enemy_air_count
        enemy_center = self.get_enemy_units_by_type(obs, units.Terran.CommandCenter)

        killed_unit_count = obs.observation.score_cumulative.killed_value_units
        killed_building_count = obs.observation.score_cumulative.killed_value_structures
        collected_minerals = obs.observation.score_cumulative.collected_minerals
        spent_minerals = obs.observation.score_cumulative.spent_minerals
        obs.observation
        idle_worker_time = obs.observation.score_cumulative.idle_worker_time
        idle_production_time = obs.observation.score_cumulative.idle_production_time

#         if(enemy_total_count > 0):
#             print(enemy_total_count)
        #         print("barracks : ", len(barrackses), " supply : ", len(supply_depots))
        #         print("free_supply : ", free_supply, " marines : ", len(marines))
        #         print("supply : ", len(supply_depots), " complete : ", len(completed_supply_depots))

        return (
            #                 len(idle_scvs),
#             len(scvs),
            len(supply_depots),
            len(barrackses),
            len(marines),
            round(obs.observation.player.minerals / 10, 0),
            round(spent_minerals / 10, 0),
            #                 too_much_minerals,
            idle_production_time,
            killed_unit_count,
            killed_building_count,
            len(enemy_center),
            len(enemy_scvs),
            len(enemy_supply_depots),
#             len(enemy_barrackses),
#             len(enemy_Factory),
#             len(enemy_Bunker),
            enemy_total_count
        )

    #         return (
    #                 len(scvs),
    # #                 len(idle_scvs),
    #                 len(supply_depots),
    #                 len(barrackses),
    #                 len(marines),
    # #                 queued_marines,
    #                 free_supply,
    # #                 can_afford_marine,
    # #                 too_much_minerals,
    # #                 collected_minerals,
    #                 spent_minerals,
    # #                 idle_production_time,
    #                 killed_unit_count,
    #                 killed_building_count,
    #                 idle_worker_time,
    #                 idle_production_time,
    #                 len(enemy_command_centers),
    #                 len(enemy_scvs),
    #                 len(enemy_supply_depots),
    #                 len(enemy_barrackses),
    #                 len(enemy_marines),
    # #                 len(enemy_Marauder),
    # #                 len(enemy_Reaper),
    #                 len(enemy_Factory),
    # #                 len(enemy_Hellion) + len(enemy_Hellbat),
    # #                 len(enemy_SiegeTank),
    # #                 len(enemy_Cyclone),
    # #                 len(enemy_WidowMine),
    # #                 len(enemy_Thor),
    #                 len(enemy_Starport),
    # #                 len(enemy_Viking),
    # #                 len(enemy_Medivac),
    # #                 len(enemy_Liberator),
    # #                 len(enemy_Raven),
    # #                 len(enemy_Battlecruiser),
    #                 enemy_land_count,
    #                 enemy_total_count
    # #                 enemy_air_count,
    #                 )

    # return (
    # len(command_centers),
    #         len(scvs),
    #         len(idle_scvs),
    #         len(supply_depots),
    #         # len(completed_supply_depots),
    #         len(barrackses),
    #         # len(completed_barrackses),
    #         len(marines),
    #         queued_marines,
    #         free_supply,
    #         # can_afford_supply_depot,
    #         # can_afford_barracks,
    #         can_afford_marine,
    #         too_much_minerals,
    #         minerals_size,
    #         len(enemy_command_centers),
    #         len(enemy_scvs),
    #         # len(enemy_idle_scvs),
    #         len(enemy_supply_depots),
    #         # len(enemy_completed_supply_depots),
    #         len(enemy_barrackses),
    #         # len(enemy_completed_barrackses),
    #         len(enemy_marines),
    #         killed_unit_count,
    #         killed_building_count
    #         )

    def step(self, obs):
        super(TerranRLAgentWithRawActsAndRawObs, self).step(obs)

        # time.sleep(0.5)

        state_org = self.get_state(obs)

        state = torch.tensor(state_org).float().view(1, self.s_dim).to(device)
        action_idx = self.dqn.choose_action(state)
        action = self.actions[action_idx]
        done = True if obs.last() else False

        if self.previous_action is not None:
            experience = (self.previous_state.to(device),
                          torch.tensor(self.previous_action).view(1, 1).to(device),
                          torch.tensor(obs.reward).view(1, 1).to(device),
                          state.to(device),
                          torch.tensor(done).view(1, 1).to(device))
            self.memory.push(experience)

        # return (
        # 0         len(command_centers),
        # 1         len(scvs),
        # 2         len(idle_scvs),
        # 3         len(supply_depots),
        # 4         len(barrackses),
        # 5         len(marines),
        # 6         queued_marines,
        # 7         free_supply,
        # 8         can_afford_marine,
        # 9         too_much_minerals,
        # 10         minerals_size,
        # 11         len(enemy_command_centers),
        # 12         len(enemy_scvs),
        # 13        len(enemy_supply_depots),
        # 14         len(enemy_barrackses),
        # 15         len(enemy_marines),
        # 16         killed_unit_count,
        # 17         killed_building_count
        # 18         )

        self.previous_state = state
        self.previous_action = action_idx
        self.cum_reward = obs.reward

        if obs.last():
            supply_depots = self.get_my_units_by_type(obs, units.Terran.SupplyDepot)
            marines = self.get_my_units_by_type(obs, units.Terran.Marine)
            barrackses = self.get_my_units_by_type(obs, units.Terran.Barracks)
            print("barracks : ", len(barrackses), " supply : ", len(supply_depots))
            print("marines : ", len(marines))
            self.episode_count = self.episode_count + 1

            if len(self.memory) >= self.init_sampling:
                # training dqn
                sampled_exps = self.memory.sample(self.batch_size)
                sampled_exps = prepare_training_inputs(sampled_exps, device)
                self.dqn.learn(*sampled_exps)

            if self.episode_count % self.target_update_interval == 0:
                self.dqn.qnet_target.load_state_dict(self.dqn.qnet.state_dict())

            if self.episode_count % self.print_every == 0:
                msg = (self.episode_count, self.cum_reward, self.epsilon)
                print("Episode : {:4.0f} | Cumulative Reward : {:4.0f} | Epsilon : {:.3f}".format(*msg))

            torch.save(self.dqn.qnet.state_dict(), self.data_file_qnet + '.pt')
            torch.save(self.dqn.qnet_target.state_dict(), self.data_file_qnet_target + '.pt')

            scores_window.append(obs.reward)  # save most recent reward
            win_rate = scores_window.count(1) / len(scores_window) * 100
            tie_rate = scores_window.count(0) / len(scores_window) * 100
            lost_rate = scores_window.count(-1) / len(scores_window) * 100

            scores.append([win_rate, tie_rate, lost_rate])  # save most recent score(win_rate, tie_rate, lost_rate)
            with open(self.score_file + '.txt', "wb") as fp:
                pickle.dump(scores, fp)
            #                 print(np.array(scores))

            # writer.add_scalar("Loss/train", self.cum_loss/obs.observation.game_loop, self.episode_count)
            # writer.add_scalar("Score", self.cum_reward, self.episode_count)

        return getattr(self, action)(obs)
    
if __name__ == "__main__":
  app.run(main)

I0924 00:05:48.254783 140688196453760 sc_process.py:135] Launching SC2: /home/jupyter/StarCraftII/Versions/Base59877/SC2_x64 -listen 127.0.0.1 -port 24377 -dataDir /home/jupyter/StarCraftII/ -tempDir /tmp/sc-3x6i_o00/
I0924 00:05:48.341655 140688196453760 remote_controller.py:167] Connecting to: ws://127.0.0.1:24377/sc2api, attempt: 0, running: True


Bot
[Agent(race=[<Race.terran: 1>], name='<unknown>'), Bot(race=[<Race.terran: 1>], difficulty=<Difficulty.medium: 3>, build=[<BotBuild.random: 1>])]


I0924 00:05:49.345455 140688196453760 remote_controller.py:167] Connecting to: ws://127.0.0.1:24377/sc2api, attempt: 1, running: True
I0924 00:05:50.348520 140688196453760 remote_controller.py:167] Connecting to: ws://127.0.0.1:24377/sc2api, attempt: 2, running: True
I0924 00:05:56.286703 140688196453760 sc2_env.py:314] Environment is ready
I0924 00:05:56.297224 140688196453760 sc2_env.py:507] Starting episode 1: [terran, terran] on Simple64
I0924 00:07:49.534119 140688196453760 sc2_env.py:725] Episode 1 finished after 16080 game steps. Outcome: [-1], reward: [-1], score: [65]


barracks :  0  supply :  0
marines :  0
Episode :    1 | Cumulative Reward :   -1 | Epsilon : 0.900


I0924 00:07:52.878456 140688196453760 sc2_env.py:507] Starting episode 2: [terran, terran] on Simple64
I0924 00:09:55.672523 140688196453760 sc2_env.py:725] Episode 2 finished after 17880 game steps. Outcome: [-1], reward: [-1], score: [260]


...

I0924 05:04:05.590346 140688196453760 sc2_env.py:507] Starting episode 166: [terran, terran] on Simple64
I0924 05:06:17.166558 140688196453760 sc2_env.py:725] Episode 166 finished after 18312 game steps. Outcome: [-1], reward: [-1], score: [470]


barracks :  0  supply :  0
marines :  0
Episode :  166 | Cumulative Reward :   -1 | Epsilon : 0.801


I0924 05:06:20.507662 140688196453760 sc2_env.py:507] Starting episode 167: [terran, terran] on Simple64
I0924 05:07:52.808553 140688196453760 sc2_env.py:725] Episode 167 finished after 14328 game steps. Outcome: [-1], reward: [-1], score: [60]


barracks :  0  supply :  0
marines :  0
Episode :  167 | Cumulative Reward :   -1 | Epsilon : 0.800


I0924 05:07:56.172769 140688196453760 sc2_env.py:507] Starting episode 168: [terran, terran] on Simple64
I0924 05:09:37.665703 140688196453760 sc2_env.py:725] Episode 168 finished after 15296 game steps. Outcome: [-1], reward: [-1], score: [120]


### [Winning rate graph]

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# SCORE_FILE = 'rlagent_with_vanilla_dqn_score'

In [ ]:
with open(SCORE_FILE + '.txt', "rb") as fp:
    scores = pickle.load(fp)

In [ ]:
np_scores = np.array(scores)
np_scores

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(np_scores)), np_scores.T[0], color='r', label='win rate')
plt.plot(np.arange(len(np_scores)), np_scores.T[1], color='g', label='tie rate')
plt.plot(np.arange(len(np_scores)), np_scores.T[2], color='b', label='lose rate')
plt.ylabel('Score %')
plt.xlabel('Episode #')
plt.legend(loc='best')
plt.show()

In [ ]:
print device